In [ ]:
!pip install sentencepiece
!pip install python-Levenshtein
#!pip install warp_rnnt
!pip install hydra-core

In [ ]:
!pip install wget

In [ ]:
!pip install jamo

In [ ]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

In [ ]:
!tar xvzf dev-clean.tar.gz
!tar xvzf test-clean.tar.gz

In [ ]:
!python -m pip install -U pip

**Codes from :** https://github.com/sooftware/kospeech

* https://github.com/sooftware/kospeech/blob/latest/LICENSE


In [1]:
import Levenshtein as Lev

In [2]:
def wer(s1,s2):
    b = set(s1.split()+s2.split())
    word_map = dict(zip(b,range(len(b))))
    print("word_map:",word_map)
    w1 = [chr(word_map[w]) for w in s1.split()]
    w2 = [chr(word_map[w]) for w in s2.split()]
    dev = max(len(s2.split()),1)
    score = Lev.distance("".join(w1),"".join(w2))/dev
    return score

def cer(s1,s2):
    w1 = s1.replace(" ","")
    w2 = s2.replace(" ","")
    dev = max(len(s2),1)
    score = Lev.distance((w1),(w2))/dev
    return score

In [3]:
# Copyright (c) 2020, Soohwan Kim. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
import torch.nn as nn
import torch.nn.init as init
from torch import Tensor


class ResidualConnectionModule(nn.Module):
    """
    Residual Connection Module.
    outputs = (module(inputs) x module_factor + inputs x input_factor)
    """
    def __init__(
            self,
            module: nn.Module,
            module_factor: float = 1.0,
            input_factor: float = 1.0,
    ) -> None:
        super(ResidualConnectionModule, self).__init__()
        self.module = module
        self.module_factor = module_factor
        self.input_factor = input_factor

    def forward(self, inputs: Tensor) -> Tensor:
        return (self.module(inputs) * self.module_factor) + (inputs * self.input_factor)


class Linear(nn.Module):
    """
    Wrapper class of torch.nn.Linear
    Weight initialize by xavier initialization and bias initialize to zeros.
    """
    def __init__(self, in_features: int, out_features: int, bias: bool = True) -> None:
        super(Linear, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        init.xavier_uniform_(self.linear.weight)
        if bias:
            init.zeros_(self.linear.bias)

    def forward(self, x: Tensor) -> Tensor:
        return self.linear(x)


class View(nn.Module):
    """ Wrapper class of torch.view() for Sequential module. """
    def __init__(self, shape: tuple, contiguous: bool = False):
        super(View, self).__init__()
        self.shape = shape
        self.contiguous = contiguous

    def forward(self, inputs):
        if self.contiguous:
            inputs = inputs.contiguous()
        return inputs.view(*self.shape)


class Transpose(nn.Module):
    """ Wrapper class of torch.transpose() for Sequential module. """
    def __init__(self, shape: tuple):
        super(Transpose, self).__init__()
        self.shape = shape

    def forward(self, inputs: Tensor):
        return inputs.transpose(*self.shape)

In [4]:
import torch.nn as nn
from torch import Tensor
from typing import Tuple


class EncoderInterface(nn.Module):
    """ Base Interface of Encoder """
    def __init__(self):
        super(EncoderInterface, self).__init__()

    def count_parameters(self) -> int:
        """ Count parameters of encoder """
        return sum([p.numel for p in self.parameters()])

    def update_dropout(self, dropout_p: float) -> None:
        """ Update dropout probability of encoder """
        for name, child in self.named_children():
            if isinstance(child, nn.Dropout):
                child.p = dropout_p

    def forward(self, inputs: Tensor, input_lengths: Tensor):
        """
        Forward propagate for encoder training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        """
        raise NotImplementedError


class BaseEncoder(EncoderInterface):
    """ ASR Encoder Super Class for KoSpeech model implementation """
    """
    supported_extractors = {
        'ds2': DeepSpeech2Extractor,
        'vgg': VGGExtractor,
    }
    """

    def __init__(
            self,
            input_dim: int,
            extractor: str = 'vgg',
            d_model: int = None,
            num_classes: int = None,
            dropout_p: float = None,
            activation: str = 'hardtanh',
            joint_ctc_attention: bool = False,
    ) -> None:
        super(BaseEncoder, self).__init__()
        if joint_ctc_attention:
            assert num_classes, "If `joint_ctc_attention` True, `num_classes` should be not None"
            assert dropout_p, "If `joint_ctc_attention` True, `dropout_p` should be not None"
            assert d_model, "If `joint_ctc_attention` True, `d_model` should be not None"

        if extractor is not None:
            extractor = self.supported_extractors[extractor.lower()]
            self.conv = extractor(input_dim=input_dim, activation=activation)

        self.conv_output_dim = self.conv.get_output_dim()
        self.num_classes = num_classes
        self.joint_ctc_attention = joint_ctc_attention

        if self.joint_ctc_attention:
            self.fc = nn.Sequential(
                nn.BatchNorm1d(d_model),
                Transpose(shape=(1, 2)),
                nn.Dropout(dropout_p),
                Linear(d_model, num_classes, bias=False),
            )

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
        """
        Forward propagate a `inputs` for  encoder training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            (Tensor, Tensor, Tensor):
            * encoder_outputs: A output sequence of encoder. `FloatTensor` of size ``(batch, seq_length, dimension)``
            * encoder_output_lengths: The length of encoder outputs. ``(batch)``
            * encoder_log_probs: Log probability of encoder outputs will be passed to CTC Loss.
                If joint_ctc_attention is False, return None.
        """
        raise NotImplementedError


class TransducerEncoder(EncoderInterface):
    """ ASR Transducer Encoder Super class for KoSpeech model implementation """
    def __init__(self):
        super(TransducerEncoder, self).__init__()

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tensor:
        """
        Forward propagate a `inputs` for  encoder training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            (Tensor, Tensor)
            * outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            * output_lengths (torch.LongTensor): The length of output tensor. ``(batch)``
        """
        raise NotImplementedError

In [5]:
import torch
import torch.nn as nn
from torch import Tensor
from typing import Tuple


class DecoderInterface(nn.Module):
    def __init__(self):
        super(DecoderInterface, self).__init__()

    def count_parameters(self) -> int:
        """ Count parameters of encoder """
        return sum([p.numel for p in self.parameters()])

    def update_dropout(self, dropout_p: float) -> None:
        """ Update dropout probability of encoder """
        for name, child in self.named_children():
            if isinstance(child, nn.Dropout):
                child.p = dropout_p


class BaseDecoder(DecoderInterface):
    """ ASR Decoder Super Class for KoSpeech model implementation """
    def __init__(self):
        super(BaseDecoder, self).__init__()

    def forward(self, targets: Tensor, encoder_outputs: Tensor, **kwargs) -> Tensor:
        """
        Forward propagate a `encoder_outputs` for training.
        Args:
            targets (torch.LongTensr): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
            encoder_outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        Returns:
            * predicted_log_probs (torch.FloatTensor): Log probability of model predictions.
        """
        raise NotImplementedError

    @torch.no_grad()
    def decode(self, encoder_outputs: Tensor, *args) -> Tensor:
        """
        Decode encoder_outputs.
        Args:
            encoder_outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        Returns:
            * predicted_log_probs (torch.FloatTensor): Log probability of model predictions.
        """
        raise NotImplementedError


class TransducerDecoder(DecoderInterface):
    """ ASR Transducer Decoder Super Class for KoSpeech model implementation """
    def __init__(self):
        super(TransducerDecoder, self).__init__()

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        """
        Forward propage a `inputs` (targets) for training.
        Args:
            inputs (torch.LongTensor): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            (Tensor, Tensor):
            * decoder_outputs (torch.FloatTensor): A output sequence of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            * hidden_states (torch.FloatTensor): A hidden state of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        """
        raise NotImplementedError

In [6]:
import torch
import torch.nn as nn
from torch import Tensor
from typing import Tuple



class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def count_parameters(self) -> int:
        """ Count parameters of encoder """
        return sum([p.numel for p in self.parameters()])

    def update_dropout(self, dropout_p: float) -> None:
        """ Update dropout probability of encoder """
        for name, child in self.named_children():
            if isinstance(child, nn.Dropout):
                child.p = dropout_p

    @torch.no_grad()
    def recognize(self, inputs: Tensor, input_lengths: Tensor):
        raise NotImplementedError


class EncoderModel(BaseModel):
    """ Super class of KoSpeech's Encoder only Models """
    def __init__(self):
        super(EncoderModel, self).__init__()
        self.decoder = None

    def set_decoder(self, decoder):
        """ Setter for decoder """
        self.decoder = decoder

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        """
        Forward propagate a `inputs` for  ctc training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            (Tensor, Tensor):
            * predicted_log_prob (torch.FloatTensor)s: Log probability of model predictions.
            * output_lengths (torch.LongTensor): The length of output tensor ``(batch)``
        """
        raise NotImplementedError

    @torch.no_grad()
    def decode(self, predicted_log_probs: Tensor) -> Tensor:
        """
        Decode encoder_outputs.
        Args:
            predicted_log_probs (torch.FloatTensor):Log probability of model predictions. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        Returns:
            * predictions (torch.FloatTensor): Result of model predictions.
        """
        return predicted_log_probs.max(-1)[1]

    @torch.no_grad()
    def recognize(self, inputs: Tensor, input_lengths: Tensor) -> Tensor:
        """
        Recognize input speech.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            * predictions (torch.FloatTensor): Result of model predictions.
        """
        predicted_log_probs, _ = self.forward(inputs, input_lengths)
        if self.decoder is not None:
            return self.decoder.decode(predicted_log_probs)
        return self.decode(predicted_log_probs)


class EncoderDecoderModel(BaseModel):
    """ Super class of KoSpeech's Encoder-Decoder Models """
    def __init__(self, encoder: BaseEncoder, decoder: BaseDecoder) -> None:
        super(EncoderDecoderModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def set_encoder(self, encoder):
        """ Setter for encoder """
        self.encoder = encoder

    def set_decoder(self, decoder):
        """ Setter for decoder """
        self.decoder = decoder

    def count_parameters(self) -> int:
        """ Count parameters of encoder """
        num_encoder_parameters = self.encoder.count_parameters()
        num_decoder_parameters = self.decoder.count_parameters()
        return num_encoder_parameters + num_decoder_parameters

    def update_dropout(self, dropout_p) -> None:
        """ Update dropout probability of model """
        self.encoder.update_dropout(dropout_p)
        self.decoder.update_dropout(dropout_p)

    def forward(
            self,
            inputs: Tensor,
            input_lengths: Tensor,
            targets: Tensor,
            *args,
    ) -> Tuple[Tensor, Tensor, Tensor]:
        """
        Forward propagate a `inputs` and `targets` pair for training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
            targets (torch.LongTensr): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
        Returns:
            (Tensor, Tensor, Tensor)
            * predicted_log_probs (torch.FloatTensor): Log probability of model predictions.
            * encoder_output_lengths: The length of encoder outputs. ``(batch)``
            * encoder_log_probs: Log probability of encoder outputs will be passed to CTC Loss.
                If joint_ctc_attention is False, return None.
        """
        raise NotImplementedError

    @torch.no_grad()
    def recognize(self, inputs: Tensor, input_lengths: Tensor) -> Tensor:
        """
        Recognize input speech. This method consists of the forward of the encoder and the decode() of the decoder.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            * predictions (torch.FloatTensor): Result of model predictions.
        """
        encoder_outputs, encoder_output_lengths, _ = self.encoder(inputs, input_lengths)
        return self.decoder.decode(encoder_outputs, encoder_output_lengths)


class TransducerModel(BaseModel):
    """ Super class of KoSpeech's Transducer Models """
    def __init__(
            self,
            encoder: TransducerEncoder,
            decoder: TransducerDecoder,
            d_model: int,
            num_classes: int,
    ) -> None:
        super(TransducerModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.fc = Linear(d_model<<1, num_classes, bias=False)

    def set_encoder(self, encoder):
        """ Setter for encoder """
        self.encoder = encoder

    def set_decoder(self, decoder):
        """ Setter for decoder """
        self.decoder = decoder

    def count_parameters(self) -> int:
        """ Count parameters of encoder """
        num_encoder_parameters = self.encoder.count_parameters()
        num_decoder_parameters = self.decoder.count_parameters()
        return num_encoder_parameters + num_decoder_parameters

    def update_dropout(self, dropout_p) -> None:
        """ Update dropout probability of model """
        self.encoder.update_dropout(dropout_p)
        self.decoder.update_dropout(dropout_p)

    def joint(self, encoder_outputs: Tensor, decoder_outputs: Tensor) -> Tensor:
        """
        Joint `encoder_outputs` and `decoder_outputs`.
        Args:
            encoder_outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            decoder_outputs (torch.FloatTensor): A output sequence of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        Returns:
            * outputs (torch.FloatTensor): outputs of joint `encoder_outputs` and `decoder_outputs`..
        """
        if encoder_outputs.dim() == 3 and decoder_outputs.dim() == 3:
            input_length = encoder_outputs.size(1)
            target_length = decoder_outputs.size(1)

            encoder_outputs = encoder_outputs.unsqueeze(2)
            decoder_outputs = decoder_outputs.unsqueeze(1)

            encoder_outputs = encoder_outputs.repeat([1, 1, target_length, 1])
            decoder_outputs = decoder_outputs.repeat([1, input_length, 1, 1])

            #print("  encoder_outputs:",encoder_outputs.shape)
            #print("  decoder_outputs:",decoder_outputs.shape)
        #outputs = (encoder_outputs + decoder_outputs)
        outputs = torch.cat((encoder_outputs, decoder_outputs), dim=-1)
        #print(" outputs:",outputs.shape)
        outputs = self.fc(outputs).log_softmax(dim=-1)
        #print(" outputs2:",outputs.shape)
        return outputs

    def forward(
            self,
            inputs: Tensor,
            input_lengths: Tensor,
            targets: Tensor,
            target_lengths: Tensor,
    ) -> Tensor:
        """
        Forward propagate a `inputs` and `targets` pair for training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
            targets (torch.LongTensr): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
            target_lengths (torch.LongTensor): The length of target tensor. ``(batch)``
        Returns:
            * predictions (torch.FloatTensor): Result of model predictions.
        """
        encoder_outputs, _ = self.encoder(inputs, input_lengths)
        decoder_outputs, _ = self.decoder(targets, target_lengths)
        return self.joint(encoder_outputs, decoder_outputs)

    @torch.no_grad()
    def decode(self, encoder_output: Tensor, max_length: int) -> Tensor:
        """
        Decode `encoder_outputs`.
        Args:
            encoder_output (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(seq_length, dimension)``
            max_length (int): max decoding time step
        Returns:
            * predicted_log_probs (torch.FloatTensor): Log probability of model predictions.
        """
        pred_tokens, hidden_state = list(), None
        decoder_input = encoder_output.new_tensor([[self.decoder.sos_id]], dtype=torch.long)

        for t in range(max_length):
            decoder_output, hidden_state = self.decoder(decoder_input, hidden_states=hidden_state)
            step_output = self.joint(encoder_output[t].view(-1), decoder_output.view(-1))
            step_output = step_output.softmax(dim=0)
            pred_token = step_output.argmax(dim=0)
            pred_token = int(pred_token.item())
            pred_tokens.append(pred_token)
            decoder_input = step_output.new_tensor([[pred_token]], dtype=torch.long)

        return torch.LongTensor(pred_tokens)

    @torch.no_grad()
    def recognize(self, inputs: Tensor, input_lengths: Tensor) -> Tensor:
        """
        Recognize input speech. This method consists of the forward of the encoder and the decode() of the decoder.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            * outputs (torch.FloatTensor): Result of model predictions.
        """
        outputs = list()

        encoder_outputs, output_lengths = self.encoder(inputs, input_lengths)
        max_length = encoder_outputs.size(1)

        for encoder_output in encoder_outputs:
            decoded_seq = self.decode(encoder_output, max_length)
            outputs.append(decoded_seq)

        outputs = torch.stack(outputs, dim=1).transpose(0, 1)

        return outputs

In [7]:
import torch.nn as nn
from torch import Tensor
from typing import Tuple


class DecoderRNNT(TransducerDecoder):
    """
    Decoder of RNN-Transducer
    Args:
        num_classes (int): number of classification
        hidden_state_dim (int, optional): hidden state dimension of decoder (default: 512)
        output_dim (int, optional): output dimension of encoder and decoder (default: 512)
        num_layers (int, optional): number of decoder layers (default: 1)
        rnn_type (str, optional): type of rnn cell (default: lstm)
        sos_id (int, optional): start of sentence identification
        eos_id (int, optional): end of sentence identification
        dropout_p (float, optional): dropout probability of decoder
    Inputs: inputs, input_lengths
        inputs (torch.LongTensor): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
        input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        hidden_states (torch.FloatTensor): A previous hidden state of decoder. `FloatTensor` of size
            ``(batch, seq_length, dimension)``
    Returns:
        (Tensor, Tensor):
        * decoder_outputs (torch.FloatTensor): A output sequence of decoder. `FloatTensor` of size
            ``(batch, seq_length, dimension)``
        * hidden_states (torch.FloatTensor): A hidden state of decoder. `FloatTensor` of size
            ``(batch, seq_length, dimension)``
    """
    supported_rnns = {
        'lstm': nn.LSTM,
        'gru': nn.GRU,
        'rnn': nn.RNN,
    }

    def __init__(
            self,
            num_classes: int,
            hidden_state_dim: int,
            output_dim: int,
            num_layers: int,
            rnn_type: str = 'lstm',
            sos_id: int = 1,
            eos_id: int = 2,
            dropout_p: float = 0.2,
    ):
        super(DecoderRNNT, self).__init__()
        self.hidden_state_dim = hidden_state_dim
        self.sos_id = sos_id
        self.eos_id = eos_id
        self.embedding = nn.Embedding(num_classes, hidden_state_dim)
        rnn_cell = self.supported_rnns[rnn_type.lower()]
        self.rnn = rnn_cell(
            input_size=hidden_state_dim,
            hidden_size=hidden_state_dim,
            num_layers=num_layers,
            bias=True,
            batch_first=True,
            dropout=dropout_p,
            bidirectional=False,
        )
        self.out_proj = Linear(hidden_state_dim, output_dim)

    def forward(
            self,
            inputs: Tensor,
            input_lengths: Tensor = None,
            hidden_states: Tensor = None,
    ) -> Tuple[Tensor, Tensor]:
        """
        Forward propage a `inputs` (targets) for training.
        Args:
            inputs (torch.LongTensor): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
            hidden_states (torch.FloatTensor): A previous hidden state of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        Returns:
            (Tensor, Tensor):
            * decoder_outputs (torch.FloatTensor): A output sequence of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            * hidden_states (torch.FloatTensor): A hidden state of decoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
        """
        embedded = self.embedding(inputs)
        #print("embeded:",embedded.shape,"hidden_states:",hidden_states)
        outputs, hidden_states = self.rnn(embedded, hidden_states)
        outputs = self.out_proj(outputs)
        return outputs, hidden_states
      

In [8]:
# Copyright (c) 2021, Soohwan Kim. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch.nn as nn
from torch import Tensor
from typing import Tuple



class EncoderRNNT(TransducerEncoder):
    """
    Encoder of RNN-Transducer.
    Args:
        input_dim (int): dimension of input vector
        hidden_state_dim (int, optional): hidden state dimension of encoder (default: 320)
        output_dim (int, optional): output dimension of encoder and decoder (default: 512)
        num_layers (int, optional): number of encoder layers (default: 4)
        rnn_type (str, optional): type of rnn cell (default: lstm)
        bidirectional (bool, optional): if True, becomes a bidirectional encoder (default: True)
    Inputs: inputs, input_lengths
        inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
            `FloatTensor` of size ``(batch, seq_length, dimension)``.
        input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
    Returns:
        (Tensor, Tensor)
        * outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
            ``(batch, seq_length, dimension)``
        * hidden_states (torch.FloatTensor): A hidden state of encoder. `FloatTensor` of size
            ``(batch, seq_length, dimension)``
    """
    supported_rnns = {
        'lstm': nn.LSTM,
        'gru': nn.GRU,
        'rnn': nn.RNN,
    }

    def __init__(
            self,
            input_dim: int,
            hidden_state_dim: int,
            output_dim: int,
            num_layers: int,
            rnn_type: str = 'lstm',
            dropout_p: float = 0.2,
            bidirectional: bool = True,
    ):
        super(EncoderRNNT, self).__init__()
        self.hidden_state_dim = hidden_state_dim
        rnn_cell = self.supported_rnns[rnn_type.lower()]
        self.rnn = rnn_cell(
            input_size=input_dim,
            hidden_size=hidden_state_dim,
            num_layers=num_layers,
            bias=True,
            batch_first=True,
            dropout=dropout_p,
            bidirectional=bidirectional,
        )
        self.out_proj = Linear(hidden_state_dim << 1 if bidirectional else hidden_state_dim, output_dim)

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        """
        Forward propagate a `inputs` for  encoder training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        Returns:
            (Tensor, Tensor)
            * outputs (torch.FloatTensor): A output sequence of encoder. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            * output_lengths (torch.LongTensor): The length of output tensor. ``(batch)``
        """
        inputs = nn.utils.rnn.pack_padded_sequence(inputs.transpose(0, 1), input_lengths.cpu())
        outputs, hidden_states = self.rnn(inputs)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = self.out_proj(outputs.transpose(0, 1))
        return outputs, input_lengths

In [9]:
# Copyright (c) 2021, Soohwan Kim. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from torch import Tensor



class RNNTransducer(TransducerModel):
    """
    RNN-Transducer are a form of sequence-to-sequence models that do not employ attention mechanisms.
    Unlike most sequence-to-sequence models, which typically need to process the entire input sequence
    (the waveform in our case) to produce an output (the sentence), the RNN-T continuously processes input samples and
    streams output symbols, a property that is welcome for speech dictation. In our implementation,
    the output symbols are the characters of the alphabet.
    Args:
        num_classes (int): number of classification
        input_dim (int): dimension of input vector
        num_encoder_layers (int, optional): number of encoder layers (default: 4)
        num_decoder_layers (int, optional): number of decoder layers (default: 1)
        encoder_hidden_state_dim (int, optional): hidden state dimension of encoder (default: 320)
        decoder_hidden_state_dim (int, optional): hidden state dimension of decoder (default: 512)
        output_dim (int, optional): output dimension of encoder and decoder (default: 512)
        rnn_type (str, optional): type of rnn cell (default: lstm)
        bidirectional (bool, optional): if True, becomes a bidirectional encoder (default: True)
        encoder_dropout_p (float, optional): dropout probability of encoder
        decoder_dropout_p (float, optional): dropout probability of decoder
        sos_id (int, optional): start of sentence identification
        eos_id (int, optional): end of sentence identification
    Inputs: inputs, input_lengths, targets, target_lengths
        inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
            `FloatTensor` of size ``(batch, seq_length, dimension)``.
        input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
        targets (torch.LongTensr): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
        target_lengths (torch.LongTensor): The length of target tensor. ``(batch)``
    Returns:
        * predictions (torch.FloatTensor): Result of model predictions.
    """
    def __init__(
            self,
            num_classes: int,
            input_dim: int,
            num_encoder_layers: int = 4,
            num_decoder_layers: int = 1,
            encoder_hidden_state_dim: int = 320,
            decoder_hidden_state_dim: int = 256,
            output_dim: int = 256,
            rnn_type: str = "lstm",
            bidirectional: bool = True,
            encoder_dropout_p: float = 0.2,
            decoder_dropout_p: float = 0.2,
            sos_id: int = 1,
            eos_id: int = 2,
    ):
        encoder = EncoderRNNT(
            input_dim=input_dim,
            hidden_state_dim=encoder_hidden_state_dim,
            output_dim=output_dim,
            num_layers=num_encoder_layers,
            rnn_type=rnn_type,
            dropout_p=encoder_dropout_p,
            bidirectional=bidirectional,
        )
        decoder = DecoderRNNT(
            num_classes=num_classes,
            hidden_state_dim=decoder_hidden_state_dim,
            output_dim=output_dim,
            num_layers=num_decoder_layers,
            rnn_type=rnn_type,
            sos_id=sos_id,
            eos_id=eos_id,
            dropout_p=decoder_dropout_p,
        )
        super(RNNTransducer, self).__init__(encoder, decoder, output_dim, num_classes)

    def forward(
            self,
            inputs: Tensor,
            input_lengths: Tensor,
            targets: Tensor,
            target_lengths: Tensor
    ) -> Tensor:
        """
        Forward propagate a `inputs` and `targets` pair for training.
        Args:
            inputs (torch.FloatTensor): A input sequence passed to encoder. Typically for inputs this will be a padded
                `FloatTensor` of size ``(batch, seq_length, dimension)``.
            input_lengths (torch.LongTensor): The length of input tensor. ``(batch)``
            targets (torch.LongTensr): A target sequence passed to decoder. `IntTensor` of size ``(batch, seq_length)``
            target_lengths (torch.LongTensor): The length of target tensor. ``(batch)``
        Returns:
            * predictions (torch.FloatTensor): Result of model predictions.
        """
        return super().forward(inputs, input_lengths, targets, target_lengths)

In [10]:
# Copyright (c) 2021, Soohwan Kim. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
import torch.nn as nn
import torchaudio


class TransducerLoss(nn.Module):
    """
    Transducer loss module.
    Args:
        blank_id (int): blank symbol id
    """

    def __init__(self, blank_id: int) -> None:
        """Construct an TransLoss object."""
        super().__init__()
        #self._device = torch.device("cuda:0" if(torch.cuda.is_available()) else "cpu")
        """
        try:
            from warp_rnnt import rnnt_loss
        except ImportError:
            raise ImportError("warp-rnnt is not installed. Please re-setup")
        """
        rnnt_loss = torchaudio.transforms.RNNTLoss(blank=blank_id,reduction='sum')
        self.rnnt_loss = rnnt_loss
        self.blank_id = blank_id

    def forward(
            self,
            log_probs: torch.FloatTensor,
            targets: torch.IntTensor,
            input_lengths: torch.IntTensor,
            target_lengths: torch.IntTensor,
    ) -> torch.FloatTensor:
        """
        Compute path-aware regularization transducer loss.
        Args:
            log_probs (torch.FloatTensor): Batch of predicted sequences (batch, maxlen_in, maxlen_out+1, odim)
            targets (torch.IntTensor): Batch of target sequences (batch, maxlen_out)
            input_lengths (torch.IntTensor): batch of lengths of predicted sequences (batch)
            target_lengths (torch.IntTensor): batch of lengths of target sequences (batch)
        Returns:
            loss (torch.FloatTensor): transducer loss
        """
        """
        return self.rnnt_loss(
            log_probs,
            targets,
            input_lengths,
            target_lengths,
            reduction="mean",
            blank=self.blank_id,
            gather=True,
        )
        """
        return self.rnnt_loss(log_probs,targets,input_lengths,target_lengths)

In [11]:
import numpy as np
#import librosa
from glob import glob
import os
import soundfile as sf
from torchaudio import transforms as audioT
import random
from pathlib import Path
class DataLoader:
    def __init__(self,batch_size=1,is_test_set=False):
        super().__init__()
        self.batch_size = batch_size
        self.is_test_set = is_test_set
        common_folder = "D:\\datasets\\LibriSpeech\\train-clean-100" #
        #common_folder = "LibriSpeech/dev-clean"#"LibriSpeech/train-clean-100"
        if(is_test_set==True):
            common_folder = "LibriSpeech/test-clean"
        transcripts_files = glob(os.path.join(common_folder,"*\\*\\*.txt"),recursive=True)
        audio_files = glob(os.path.join(common_folder,"*\\*\\*.flac"),recursive=True)
        if(is_test_set==False):
            audio_files_len = len(audio_files)
            audio_files = audio_files[:audio_files_len//8]
        print("transcripts_files:",len(transcripts_files))
        print("audio_files:",len(audio_files))
        transcripts = {}
        for ts_fpath in transcripts_files:
            with open(ts_fpath,"r") as fp:
                txt = fp.read().split("\n")
                for ln in txt:
                    ln=ln.strip()
                    if(ln==""):
                        continue
                  
                    lines = ln.split(" ",1)
                    fn = lines[0]
                    lbls = lines[1].upper()
                    transcripts[fn] = lbls
        self.audio_files = audio_files
        self.transcript_keys = list(transcripts.keys())
        self.transcript_table = "".join([chr(ord("A")+i) for i in range(26)])+(" .'!?")
        #print("transcript",transcripts)
        #print("transcript keys:",self.transcript_keys)
        
        self.transcripts = transcripts
        self._audio_file_len = len(self.audio_files)
        if(is_test_set==False):
            self.shuffleDataset()

    def __len__(self):
        return len(self.audio_files)//self.batch_size
    def _loadFeature(self,fpath):
        rawinput,sample_rate = torchaudio.load(fpath) #sf.read(fpath)
        audio_transform = audioT.MFCC(sample_rate=sample_rate,n_mfcc=96,log_mels=True)
        mfcc = audio_transform(rawinput)
        #mfcc = mfcc.squeeze(dim=0)#.transpose(1,0)
        #mfcc = torch.nn.functional.pad(mfcc,pad=(0,4))
        mfcc = mfcc.squeeze(dim=0).transpose(1,0)
        #print("load mfcc:",mfcc.shape)
        #rawinput = abs(rawinput)
        #mfcc = librosa.feature.mfcc(y=rawinput,sr=sample_rate,n_mfcc=96).T
        #mfcc_mean = np.mean(mfcc)
        #mfcc_std = np.std(mfcc)
        #mfcc = (mfcc-mfcc_mean)/mfcc_std
        #mfcc = np.pad(mfcc,((0,4),(0,0)))# emformer rnn-t 가 이전 문맥과 자연스럽게 연결짓기 위해서 right context padding 을 가짐
        return mfcc
    def _convertTranscript2(self,text_list,with_blank=False):
        #transcript_table: "".join([chr(ord("A")+i) for i in range(26)])+(" .'!?")
        #               => [A~Z] & " .'!?"
        result = []
        for char in text_list:
            result.append(self.transcript_table.find(char)+3)
            if(with_blank==True):
                result.append(2)
        return result
    def shuffleDataset(self):
        random.shuffle(self.audio_files)
    def token2String(self,token_list,with_visible_tag=True):
        result = ""
        max_len = len(self.transcript_table)+2
        for i in range(len(token_list)):
            token_id = token_list[i]
            if(token_id==1):
                result+='<s>' if(with_visible_tag==True) else ""
            elif(token_id==0):
                result += '</s>' if(with_visible_tag==True) else ""
            elif(token_id==2):
                result += '</b>' if(with_visible_tag==True) else ""
            elif(2<token_id<max_len):
                result+= self.transcript_table[token_id-3]
            else:
                result+='</u?>' if(with_visible_tag==True) else ""
        return result
    
    def __getitem__(self,idx):
        if(idx>=self.__len__()):
            raise StopIteration()
        start_index,end_cond = (idx*self.batch_size,min((idx+1)*self.batch_size,(self._audio_file_len)))
        #print((start_index,end_cond))
        specs = []
        transcripts = []
        transcript_lengths = []
        specs_lengths = []

        for i in range(start_index,end_cond):
            fpath = self.audio_files[i]
            feature = self._loadFeature(fpath)
            fn = Path(fpath).stem
            specs.append(feature)#torch.from_numpy(feature))
            specs_lengths.append(feature.size(0))
            transcript_indices = [0]+self._convertTranscript2(self.transcripts[fn],with_blank=False)#+[2]
            transcripts.append(torch.from_numpy(np.array(transcript_indices)))
            transcript_lengths.append(len(transcript_indices))
        transcript_lengths = np.array(transcript_lengths)
        specs_lengths = np.array(specs_lengths)
        return specs,specs_lengths,transcripts,transcript_lengths

In [ ]:

chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

ENGS = tuple([chr(ord("A")+i) for i in range(26)])

special_chars = tuple(list(" #$!?@~^&*()[]'~.\"`_:"))

import re

def replaceBracket(sentence):
    rst = re.sub( "(\(((\S|\s)*)(\)_))","",sentence)
    rst = re.sub( "(\(((\S|\s)*)(\)))","",rst)
    return rst

def getHangeulIndex(one_character):
    
    for i,comp in enumerate(chosung+jungsung+jongsung+ENGS+special_chars):
        
        if(one_character.upper()==comp):
            result = i
            if(i<len(chosung)):
                result = i
            elif(i<len(chosung)+len(jungsung)):
                result = i+len(chosung)
            elif(i<len(chosung)+len(jungsung)+len(jongsung)):
                result = i+len(chosung)+len(jungsung)
            return i+5
    return 4
def convertString2HangeulIndex(text):
    result = []
    for character in text:
        result.append(getHangeulIndex(character))
    return result

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung],-1)
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong],-1)

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

def special_filter(sentence):
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))

def bracketContentFilter(text):
    #괄호가 페어를 이루고 있다는 전제로 짜여진 함수
    def contentExtractor(tmp_value):
        if(tmp_value.find(":")):
            tmp2 = tmp_value.split(":",1)
            return tmp2[-1]
        else:
            return tmp_value
    isBracket = False
    result = ""
    tmp = ""
    for c in text:
        if(c=="("):
            isBracket = True
        elif(c==")"):
            result += contentExtractor(tmp)
            isBracket = False
            tmp = ""
        elif(isBracket==True):
            tmp += c
        else:
            result += c
    return result

def text2vector(inputtext):
    def chosung2index(v):
        for i,cho in enumerate(chosung):
            if(v==cho):
                return i
        return -1
    def jungsung2index(v):
        for i,jung in enumerate(jungsung):
            if(jung==v):
                return i+len(chosung)
        return -1
    def jongsung2index(v):
        for i,jong in enumerate(jongsung):
            if(jong==v):
                return i+len(chosung)+len(jungsung)
        return -1
    chojongsung = list(set(chosung+jungsung))
    def chojongsung2index(v):
        for i,jong in enumerate(chojongsung):
            if(jong==v):
                return i+len(chosung)+len(jungsung)
        return -1
    
    result = []
    etc_chars = list(ENGS+special_chars)
    for i in inputtext:
        if isHangeul(i) == True:
            hexp = hangeulExplode(i)
            hexp_len = len(hexp)
            if(hexp_len==4):
                cho,jung,jong,last = hexp
                choidx = chojongsung2index(cho)#chosung2index(cho)
                if(choidx!=-1):
                    result.append(choidx)
                jungidx = chojongsung2index(jung)#jungsung2index(jung)
                if(jungidx!=-1):
                    result.append(jungidx)
                jongidx= chojongsung2index(jong)#jongsung2index(jong)
                if(jongidx!=-1):
                    result.append(jongidx)
                
            else:
                cho,jung,last = hexp
                choidx = chojongsung2index(cho)#chosung2index(cho)
                if(choidx!=-1):
                    result.append(choidx)
                jungidx = chojongsung2index(jung)#jungsung2index(jung)
                if(jungidx!=-1):
                    result.append(jungidx)
            #result.append(last)
        else:
            idx = -1
            for k,v in enumerate(etc_chars):
                if(v.lower()==i.lower()):
                    result.append(k+len(chosung)+len(jungsung)+len(jongsung))
            if(idx!=-1):
                result.append(idx)
    result = [i+4 for i in result]
    return result

def vector2text(vec):
    etc_chars = list(ENGS+special_chars)
    result = ""
    len_cho = len(chosung)
    len_jung = len(jungsung)
    len_jong = len(jongsung)
    len_etcchars = len(etc_chars)
    for i,v in enumerate(vec):
        if(v < 2 or v==3):
            continue
        if(v==2):
            break
        """
        if(v<len_cho+4):#chosung
            result+=chosung[v-4]
        elif(v<len_cho+len_jung+4):#jungsung
            result += jungsung[(v-4-len_cho)]
        elif(v<len_cho+len_jung+len_jong+4):#jongsung
            result += jongsung[(v-len_cho-len_jung-4)]
        elif(v<len_cho+len_jung+len_jong+len_etcchars+4):
            result += etc_chars[v-len_cho-len_jung-len_jong-4]
        """
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)

In [37]:
import numpy as np
import os
from glob import glob
import json
from pathlib import Path
import torch
import torchaudio
from torchaudio import transforms as audioT



class DatasetLoader:
    def __init__(self,is_train_dataset:bool,batch_size,top_dataset_folder = "자유대화 음성(일반남녀)"):
        super().__init__()
        section_folder = "Training"
        if(is_train_dataset==False):
            section_folder = "Validation"
        top_folder = os.path.join(top_dataset_folder,section_folder)
        tmp_audios = glob(os.path.join(top_folder,"*\\*.wav"))
        audios = []
        for aud in tmp_audios:
            ffolder = os.path.dirname(aud)
            fname = os.path.basename(aud)
            fn = Path(fname).stem
            trans_path = os.path.join(ffolder,fn+".json")
            if(os.path.exists(trans_path)==True):
                trans = self.__getTranscription(trans_path)
                trans = sentence_filter(trans)
                trans = replaceBracket(trans)
                #print("transcript2:",transcription)
                #trans = text2vector(trans)
                #trans = replaceBracket(trans)
                if(len(trans)>0):
                    audios.append(aud)
        self.__audios = audios
        use_subset = True
        if(is_train_dataset==True and use_subset==True):
            audio_len = len(audios)
            self.__audios = self.__audios[:audio_len//7]
        self.__audio_len = len(self.__audios)
        self._batch_size = batch_size
    def __len__(self):
        return self.__audio_len//self._batch_size
    def __getTranscription(self,fpath):
        with open(fpath,"r",encoding="UTF-8") as fp:
            data = json.load(fp)["발화정보"]["stt"]
            #data = sentence_filter(data)
            #data = jamo.h2j(data)
            #data = jamo.j2hcj(data)
        return data
    def __getRawAudioData(self,fpath,with_pytorch=True):
        if(with_pytorch==True):
            data,sample_rate = torchaudio.load(fpath)
            return data,sample_rate
        else:
            import librosa
            data,sample_rate = librosa.load(fpath)
            return data,sample_rate

    def __getAudioData(self,fpath,as_raw_data:bool = False,with_pytorch=True):
        data,sample_rate = self.__getRawAudioData(fpath,with_pytorch)
        if(as_raw_data==False):
            if(with_pytorch==True):
                mfcc_transforms = audioT.MFCC(sample_rate=sample_rate,n_mfcc=96)
                data = mfcc_transforms(data)
                data = data.transpose(1,2)
                data = data.squeeze(0)
                length = data.size(0)

            else:
                import librosa
                data= librosa.feature.mfcc(y=data,sr=sample_rate).T
                data = torch.from_numpy(data)
                length = data.size(0)
        return data,length
        
        
    def shuffleDataset(self):
        random.shuffle(self.__audios)
    def __getitem__(self,idx):
        if(idx>=self.__len__()):
            raise StopIteration()
        st_idx,ed_idx = idx*self._batch_size,min((idx+1)*self._batch_size,self.__audio_len)
        audios = []
        audio_lengths = []
        raw_transcripts = []
        transcripts = []
        transcript_length = []
        for i in range(st_idx,ed_idx):
            fpath = self.__audios[i]
            fname = os.path.basename(fpath)
            #print(fname)
            ffolder = os.path.dirname(fpath)
            transcript_fpath = os.path.join(ffolder,Path(fname).stem+".json")
            transcription = self.__getTranscription(transcript_fpath)
            audio_data,data_length = self.__getAudioData(fpath)
            audios.append(audio_data)
            audio_lengths.append(data_length)
            ori_trans = transcription
            
            transcription = bracketContentFilter(transcription)
            
            #print("transcript:",transcription)
            transcription = sentence_filter(transcription)
            
            raw_transcripts.append((ori_trans,transcription,fname))
            transcription = replaceBracket(transcription)
            #print("transcript2:",transcription)
            
            transcription = [0]+text2vector(transcription)
            #print("transcription:",transcription)
            transcripts.append(torch.from_numpy(np.array(transcription)))

            transcript_length.append(len(transcription))#temp length
        audios = torch.nn.utils.rnn.pad_sequence(audios,batch_first=True)
        audio_lengths = np.array(audio_lengths)
        transcript_length = np.array(transcript_length)
        #transcripts = np.array(transcripts)
        return audios,audio_lengths,transcripts,transcript_length#,raw_transcripts
        
if(__name__=="__main__"):
    debug_dataset = False
    if(debug_dataset==True):
        dataset_loader = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
        for i, (audios,audio_lengths,transcripts,transcript_length,raw_transcripts) in enumerate(dataset_loader):
            #print("audios:",len(audios),"transcripts:",len(transcripts))
            sample_audio = audios[0]
            sample_transcripts = transcripts[0]
            #print(sample_audio.shape)
            #print("  ",sample_transcripts)
            #labels = nn.utils.rnn.pad_sequence(transcripts,batch_first=True)
            #print("label:",labels.shape)
            label = raw_transcripts[0]
            if(1==1):#label[0].find("(")!=-1):
                filtered_label = label[1]
                print(str(i).zfill(6),f"'{label[0]}'")
                print(f"\t\t'{label[1]}'")
                print(f"\t\t\t{label[2]}")
            #if(i>10):
            #    break


In [31]:
def predict(net,dataset):
    net.eval()
    state = None
    with torch.no_grad():
        for (features,feat_lens,labels,label_lens) in dataset:
            #features = torch.from_numpy(features)
            feat_lens = torch.from_numpy(feat_lens)
            #labels = torch.from_numpy(labels)
            label_lens = torch.from_numpy(label_lens)
            features = nn.utils.rnn.pad_sequence(features,batch_first=True)#.unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
            
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            print(features.shape,labels.shape)
            output = net.recognize(features.float(),feat_lens)  # (batch, time, n_class)
            print("output:",output.shape)
            labels = labels.cpu()
            print("\torigin:",vector2text(labels[0].numpy().tolist()))
            print("\tresult:",vector2text(output[0].cpu().numpy().tolist()))
            #output = output.argmax(dim=-1)
            print("\toriginal vec:",labels)
            print("raw output:",output[0])
            output = output.numpy().tolist()
            
            """
            features = features.cpu()
            labels = labels.cpu()
            feat_lens = feat_lens.cpu()
            label_lens = label_lens.cpu()
            print("-"*10)
            print("\toriginal:",labels)
            #with_visible_tag
            ori_string = dataset.token2String(labels.numpy()[0].tolist())
            pred_string = dataset.token2String(output[0])
            print("\tori string: ",ori_string)
            print("\toutput result:",output[0][0])
            print("\topt string: ",pred_string)
            ori_string = dataset.token2String(labels.numpy()[0].tolist(),with_visible_tag=False)
            pred_string = dataset.token2String(output[0],with_visible_tag=False)
            print(f"\topt' string: '{pred_string}'")
            print("\twer:",wer(pred_string,ori_string))
            print("\tcer:",cer(pred_string,ori_string))
            """
  

In [15]:
device = torch.device("cuda:0" if(torch.cuda.is_available()==True) else "cpu")

In [16]:
net = RNNTransducer(num_classes=200,input_dim=96)
net = net.to(device)

C:\Users\imscs\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\modules\rnn.py:70: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [36]:
from torch import optim
initial_learning_rate = 0.0001
optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.95) #.CosineAnnealingLR(optimizer,T_max=100,eta_min=0.001)

In [35]:
print(scheduler.get_last_lr())

[1.705151806686164e-05]


In [41]:
predict(net,testdataset)

torch.Size([1, 381, 96]) torch.Size([1, 15])
output: torch.Size([1, 381])
	origin: ㅇㅣ ㄷㅐㄹㅣ ㅇㅏㄴㅣㅇㅑ
	result: ㅇㅓㄸㅓㄹ
	original vec: tensor([[ 0, 15, 43, 98,  7, 24,  9, 43, 98, 15, 23,  6, 43, 15, 25]],
       dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  8, 27, 52,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,

output: torch.Size([1, 581])
	origin: ㄱㅡㄹㅐㅅㅓ ㅇㅏㅊㅣㅁㅂㅏㅂㅇㅡㄴ ㅍㅗㄱㅣㅎㅏㄱㅗ ㅅㅏㅁㅅㅣㅂ ㅂㅜㄴ ㅇㅣㄹㅉㅣㄱ ㅊㅜㄹㄱㅡㄴㅎㅏㄱㅣㄹㅗ ㅎㅐㅆㅇㅓㅇㅛ
	result: ㅇㅓㄴ
	original vec: tensor([[ 0,  4, 41,  9, 24, 13, 27, 98, 15, 23, 18, 43, 60, 11, 23, 61, 15, 41,
         48, 98, 21, 31,  4, 43, 22, 23,  4, 31, 98, 13, 23, 60, 13, 43, 61, 98,
         11, 36, 48, 98, 15, 43, 52, 17, 43, 45, 98, 18, 36, 52,  4, 41, 48, 22,
         23,  4, 43,  9, 31, 98, 22, 24, 64, 15, 27, 15, 35]],
       dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,

output: torch.Size([1, 401])
	origin: ㄷㅏㄷㅡㄹ ㅁㅏㅊㅏㄴㄱㅏㅈㅣㅈㅛ
	result: ㅇㅓㄴㅡ
	original vec: tensor([[ 0,  7, 23,  7, 41, 52, 98, 10, 23, 18, 23, 48,  4, 23, 16, 43, 16, 35]],
       dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  6, 41,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

output: torch.Size([1, 521])
	origin: ㄴㅐㄱㅏ ㅁㅝㄹㅏㄷㅗ ㅈㅜㄹㅇㅣㅈㅣ ㅇㅏㄶㅇㅡㅁㅕㄴ ㅈㅜㄹㅇㅣㄹ ㄷㅔㄱㅏ ㅇㅓㅄㅈㅣ
	result: ㅇㅓㄸ
	original vec: tensor([[ 0,  6, 24,  4, 23, 98, 10, 37,  9, 23,  7, 31, 98, 16, 36, 52, 15, 43,
         16, 43, 98, 15, 23, 50, 15, 41, 10, 29, 48, 98, 16, 36, 52, 15, 43, 52,
         98,  7, 28,  4, 23, 98, 15, 27, 62, 16, 43]], dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

output: torch.Size([1, 481])
	origin: ㄱㅘㅇㅇㅕㄱ ㅇㅏㄹㄸㅡㄹ ㄱㅛㅌㅗㅇㅋㅏㄷㅡㄴㅡㄴ ㅆㅡㄱㅗ ㅇㅣㅆㅈㅣ
	result: ㅇㅓㄸㅓㄹ
	original vec: tensor([[ 0,  4, 32, 65, 15, 29, 45, 98, 15, 23, 52,  8, 41, 52, 98,  4, 35, 20,
         31, 65, 19, 23,  7, 41,  6, 41, 48, 98, 14, 41,  4, 31, 98, 15, 43, 64,
         16, 43]], dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  8, 27, 52,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0

...

output: torch.Size([1, 601])
	origin: ㄱㅡㅂㅎㅏㄱㅔ ㅂㅏㅂㅁㅏㄴ ㅁㅓㄱㄱㅗ ㅇㅘㅅㅓ ㅈㅏㄹㅣㅇㅔ ㅇㅏㄵㅇㅡㄴㅣ ㅈㅏㅁㅇㅣ ㅇㅏㄴ ㅇㅗㄹ ㅅㅜㄱㅏ ㅇㅓㅄㅈㅛ
	result: ㅇㅓㄴㅡ
	original vec: tensor([[ 0,  4, 41, 61, 22, 23,  4, 28, 98, 11, 23, 61, 10, 23, 48, 98, 10, 27,
         45,  4, 31, 98, 15, 32, 13, 27, 98, 16, 23,  9, 43, 15, 28, 98, 15, 23,
         49, 15, 41,  6, 43, 98, 16, 23, 60, 15, 43, 98, 15, 23, 48, 98, 15, 31,
         52, 98, 13, 36,  4, 23, 98, 15, 27, 62, 16, 35]], dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  6, 41,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

output: torch.Size([1, 461])
	origin: ㄱㅡㄹㅓㄴㄷㅔ ㅈㅗㅁ ㅇㅟㅎㅓㅁㅎㅏㄹ ㅅㅜㄷㅗ ㅇㅣㅆㄱㅔㅆㅇㅓ
	result: ㅇㅓㄸ
	original vec: tensor([[ 0,  4, 41,  9, 27, 48,  7, 28, 98, 16, 31, 60, 98, 15, 39, 22, 27, 60,
         22, 23, 52, 98, 13, 36,  7, 31, 98, 15, 43, 64,  4, 28, 64, 15, 27]],
       dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

output: torch.Size([1, 401])
	origin: ㄱㅡㄱㅔ ㅇㅙ ㅇㅟㅎㅓㅁㅎㅐㅇㅛ
	result: ㅇㅓㄸㅓㄹ
	original vec: tensor([[ 0,  4, 41,  4, 28, 98, 15, 33, 98, 15, 39, 22, 27, 60, 22, 24, 15, 35]],
       dtype=torch.int32)
raw output: tensor([ 0, 15, 27,  8, 27, 52,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

raw output: tensor([ 0, 15, 27,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 

KeyboardInterrupt: 

In [42]:
torch.save(net.state_dict(),"kornnt_ko.pth")

In [19]:
weights = torch.load("./kornnt_ko.pth")
net.load_state_dict(weights)

<All keys matched successfully>

In [39]:
traindataset = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
testdataset = DatasetLoader(is_train_dataset=False,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    

In [40]:
if(__name__=="__main__"):
    from tqdm import tqdm
    #initial_learning_rate = 0.0001
    #import nemo.collections.asr as nemo_asr

    max_num_epoch = 100
    #optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
    #criterion = audioT.RNNTLoss(blank=0,fused_log_softmax=True,reduction='sum')#,clamp=1.0)#'sum')
    criterion =  TransducerLoss(blank_id=0)#nemo_asr.losses.rnnt.RNNTLoss(num_classes=0)
    #traindataset = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    #testdataset = DatasetLoader(is_train_dataset=False,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    #torch.save(net.parameters(),"rnnt.pth")
    #predict(net,testdataset)
    
    for epoch in range(max_num_epoch):
        index = 0
        traindataset.shuffleDataset()
        losses = []
        net.train()
        for (features,feat_lens,labels,label_lens) in tqdm((traindataset)):
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)

            features = nn.utils.rnn.pad_sequence(features,batch_first=True)#.unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)

            #"""
            sorted_lens,indices = torch.sort(feat_lens.view(-1),dim=0,descending=True)
            features = features[indices]
            labels = labels[indices]
            feat_lens = sorted_lens
            label_lens = label_lens[indices]
            #"""
            
            features = features.float()
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            optimizer.zero_grad()
            #print(feat_lens,label_lens,labels.shape)
            #print("features:",features.shape,feat_lens)
            #print("targets:",labels.shape,label_lens)

            #joint_outputs,src_length,tgt_length,*_ = net(features.float(),feat_lens,labels,label_lens)
            joint_outputs = net(features.float(),feat_lens,labels.int(),label_lens)
            
            outputs = joint_outputs#
            #print("outputs:",outputs.shape)
            #outputs = nn.functional.log_softmax(joint_outputs,dim=-1)

            loss = criterion(log_probs=outputs,targets=(labels[...,1:]).int(),input_lengths=(feat_lens).int() ,target_lengths=(label_lens-1).int())
            del features,labels,feat_lens,label_lens
            torch.cuda.empty_cache()
            
            loss.backward()
            
            optimizer.step()
            losses.append(loss.item())
            if(index%100==0):
                print(loss.item())
            if(index%10==0):
                torch.save(net.state_dict(),"kornnt_ko.pth")
            index+=1
            
        print(f"ep={epoch}] loss= ",np.mean(losses))
        scheduler.step()
    torch.save(net.state_dict(),"kornnt_ko.pth")
    #predict(net,testdataset)


  0%|                                                                                         | 0/5704 [00:00<?, ?it/s]

1.3112173080444336


  2%|█▍                                                                             | 100/5704 [00:22<15:22,  6.07it/s]

4.981128692626953


  4%|██▊                                                                            | 200/5704 [00:43<15:30,  5.91it/s]

...

10.07901668548584


  5%|████▏                                                                          | 300/5704 [01:04<15:33,  5.79it/s]

7.918284893035889


  5%|████▎                                                                          | 311/5704 [01:06<19:18,  4.65it/s]


KeyboardInterrupt: 

In [ ]:
predict(net,testdataset)

Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Work" shall mean the work of authorship, whether in Source or
      Object form, made available under the License, as indicated by a
      copyright notice that is included in or attached to the work
      (an example is provided in the Appendix below).

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For the purposes
      of this License, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of,
      the Work and Derivative Works thereof.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean Licensor and any individual or Legal Entity
      on behalf of whom a Contribution has been received by Licensor and
      subsequently incorporated within the Work.

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensable
      by such Contributor that are necessarily infringed by their
      Contribution(s) alone or by combination of their Contribution(s)
      with the Work to which such Contribution(s) was submitted. If You
      institute patent litigation against any entity (including a
      cross-claim or counterclaim in a lawsuit) alleging that the Work
      or a Contribution incorporated within the Work constitutes direct
      or contributory patent infringement, then any patent licenses
      granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the
      Work or Derivative Works thereof in any medium, with or without
      modifications, and in Source or Object form, provided that You
      meet the following conditions:

      (a) You must give any other recipients of the Work or
          Derivative Works a copy of this License; and

      (b) You must cause any modified files to carry prominent notices
          stating that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works
          that You distribute, all copyright, patent, trademark, and
          attribution notices from the Source form of the Work,
          excluding those notices that do not pertain to any part of
          the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and
          do not modify the License. You may add Your own attribution
          notices within Derivative Works that You distribute, alongside
          or as an addendum to the NOTICE text from the Work, provided
          that such additional attribution notices cannot be construed
          as modifying the License.

      You may add Your own copyright statement to Your modifications and
      may provide additional or different license terms and conditions
      for use, reproduction, or distribution of Your modifications, or
      for any such Derivative Works as a whole, provided Your use,
      reproduction, and distribution of the Work otherwise complies with
      the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work
      by You to the Licensor shall be under the terms and conditions of
      this License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary use in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each
      Contributor provides its Contributions) on an "AS IS" BASIS,
      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
      implied, including, without limitation, any warranties or conditions
      of TITLE, NON-INFRINGEMENT, MERCHANTABILITY, or FITNESS FOR A
      PARTICULAR PURPOSE. You are solely responsible for determining the
      appropriateness of using or redistributing the Work and assume any
      risks associated with Your exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   APPENDIX: How to apply the Apache License to your work.

      To apply the Apache License to your work, attach the following
      boilerplate notice, with the fields enclosed by brackets "[]"
      replaced with your own identifying information. (Don't include
      the brackets!)  The text should be enclosed in the appropriate
      comment syntax for the file format. We also recommend that a
      file or class name and description of purpose be included on the
      same "printed page" as the copyright notice for easier
      identification within third-party archives.

   Copyright [yyyy] [name of copyright owner]

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.